In [19]:
import os
# os.chdir("/lustre/home/pintosanel/mlhat/src")  
os.chdir("/Users/llns2/Library/CloudStorage/OneDrive-UniversidadedoPorto/Mestrado/MECD/Tese/mlhat_Lara/src")  

print(os.getcwd())  

/Users/llns2/Library/CloudStorage/OneDrive-UniversidadedoPorto/Mestrado/MECD/Tese/mlhat_Lara/src


In [ ]:
from datasetsmultioutput import *
from multioutput import MLHAT
import random
from river import metrics
from river.metrics import F1
from river.metrics.base import Metrics
from river.metrics.multioutput import ExactMatch, MacroAverage, MicroAverage
import evaluate

import pandas as pd
from river import ensemble

from collections import defaultdict
import os
import csv

import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool

from collections import Counter


RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
from odac2 import ODAC2  # Import the class from the .py file


In [ ]:
import collections
import river.base
import numpy as np


class MajorityLabelset(river.base.Estimator):
    """
    Majority Labelset classifier for multi-label classification.
    Each observed labelset (e.g., [0,0,1,1,0,0]) is treated as a single class value.
    The classifier always predicts the most frequently observed labelset.
    """

    def __init__(self):
        self.vector_counts = {}  # Dictionary to store labelset frequencies
        self.majority_labelset = None
        self.max_value = -1.0

    def learn_one(self, x, y):
        """Trains the classifier on a single instance."""
        y_tuple = tuple(y.values())  # Convert dictionary to tuple for hashing

        # Count occurrences of each labelset
        freq = self.vector_counts.get(y_tuple, 0) + 1
        self.vector_counts[y_tuple] = freq

        # Update majority labelset if necessary
        if freq > self.max_value:
            self.max_value = freq
            self.majority_labelset = y_tuple

    def predict_one(self, x):
        """Returns the most frequently observed labelset."""
        if self.majority_labelset is None:
            return {label: 0.0 for label in x}  # Default to all zeros if no data has been seen

        return {label: float(val) for label, val in zip(x.keys(), self.majority_labelset)}

    def get_purpose_string(self):
        """Returns a description of the classifier."""
        return "Majority labelset classifier: always predicts the label vector most frequently seen so far."

    def reset_learning(self):
        """Resets the classifier to its initial state."""
        self.vector_counts.clear()
        self.majority_labelset = None
        self.max_value = -1.0

    def get_model_measurements(self):
        """Returns model measurements (currently not implemented)."""
        return None

    def is_randomizable(self):
        """Indicates whether the classifier has a random component (it does not)."""
        return False

    def get_model_description(self):
        """Returns a textual description of the model."""
        return str(self.majority_labelset) if self.majority_labelset else "No data seen yet."

    def get_frequency_table(self):
        """Returns the frequency table of labelsets."""
        return self.vector_counts





from collections import defaultdict
import itertools

def compute_co_occurrence(freq_table):
    """
    Computes co-occurrence frequency of individual labels and label pairs.
    
    Parameters:
    - freq_table (dict): Majority labelset frequency table.
    
    Returns:
    - co_occurrence_count (dict): Dictionary counting co-occurrences of label pairs.
    """
    co_occurrence_count = defaultdict(int)
    
    for labelset, freq in freq_table.items():
        # Convert labelset to index positions where labels are True
        active_labels = [j for j, val in enumerate(labelset) if val]

        # Count individual label occurrences
        for label in active_labels:
            co_occurrence_count[(label,)] += freq
        
        # Count pairwise and higher-order co-occurrences
        for pair in itertools.combinations(active_labels, 2):  # Change 2 to 3 for triplets
            co_occurrence_count[pair] += freq
    
    return co_occurrence_count

In [ ]:
def analyze_labelset_sizes(freq_table):
    """Extracts the distribution of labelset sizes."""
    size_distribution = Counter()
    for labelset, freq in freq_table.items():
        label_count = sum(labelset)  
        size_distribution[label_count] += freq
    return size_distribution

def get_safe_min_cluster_size(size_distribution):
    """Determines a safe minimum cluster size based on frequency distribution."""
    if not size_distribution:
        return 1
    return max(size_distribution, key=size_distribution.get) 

In [ ]:
def merge_and_split_clusters(odac2, cluster_dict, min_size, max_size, output_dir, i, color_list):
   
    merged_by_level ={}
    
    assigned_labels = set()
    color_idx = 0
    cluster_colors = {}
    
    clusters_with_level = [(path, info['members'], info['level'], info['parent'], info['siblings']) 
                          for path, info in cluster_dict.items()]
    
    
    max_iterations = 10  # Safety to prevent infinite loops
    iteration = 0
    
    # Force aggressive merging if needed
    force_merge_all_small = False
    
    while iteration < max_iterations:
        iteration += 1
        print(f"Starting iteration {iteration}")
        
        # Identify small clusters in this iteration
        small_clusters = [(path, members, level, parent, siblings) 
                         for path, members, level, parent, siblings in clusters_with_level 
                         if len(members) < min_size and not any(m in assigned_labels for m in members)]
        
        if not small_clusters:
            # print("No more small clusters found. Merging complete.")
            break
            
        print(f"Found {len(small_clusters)} small clusters in iteration {iteration}")
        
        # Group small clusters by level
        clusters_by_level = {}
        for path, members, level, parent, siblings in small_clusters:
            if level not in clusters_by_level:
                clusters_by_level[level] = []
            clusters_by_level[level].append((path, members, parent, siblings))
        
        # Sort levels to process deepest levels first (higher level number)
        levels = sorted(clusters_by_level.keys(), reverse=True)
        
        merged_any = False  
        merged_by_level = {}  
        
        if iteration > max_iterations/2 and small_clusters:
            # If we're in later iterations and still have small clusters, 
            # be more aggressive with merging
            force_merge_all_small = True
        
        # Process each level
        for level in levels:
            level_clusters = clusters_by_level[level]
            
            # First, identify true sibling pairs using path information
            sibling_pairs = []
            processed_paths = set()
            
            for path1, members1, parent1, siblings1 in level_clusters:
                if path1 in processed_paths:
                    continue
                    
                # Find siblings of this cluster
                for sibling_path in siblings1:
                    sibling_entry = next((entry for entry in level_clusters if entry[0] == sibling_path 
                                         and sibling_path not in processed_paths), None)
                    if sibling_entry:
                        # Found a small sibling - create a pair
                        sibling_pair = ((path1, members1), (sibling_entry[0], sibling_entry[1]))
                        sibling_pairs.append(sibling_pair)
                        processed_paths.add(path1)
                        processed_paths.add(sibling_path)
                        break
            
            # Process sibling pairs at this level
            for (path1, members1), (path2, members2) in sibling_pairs:
                merged_members = members1.union(members2)
                
                # print(f"Merging true sibling clusters: {path1} and {path2}")
              
                assigned_color = color_list[color_idx]
                color_idx = (color_idx + 1) % len(color_list)
                
                cluster_colors[tuple(sorted(members1))] = assigned_color
                cluster_colors[tuple(sorted(members2))] = assigned_color
                cluster_colors[tuple(sorted(merged_members))] = assigned_color
                
                if level not in merged_by_level:
                    merged_by_level[level] = []
                merged_by_level[level].append(list(merged_members))
                
                # Mark these labels as assigned
                assigned_labels.update(merged_members)
                merged_any = True
            
            # Handle remaining small clusters at this level that had no siblings
            remaining_small_clusters = [
                (path, members, parent) 
                for path, members, parent, _ in level_clusters 
                if path not in processed_paths
            ]
            
            # Try to merge remaining small clusters with their parent's other children
            for path, members, parent_path in remaining_small_clusters:
                if any(m in assigned_labels for m in members):
                    continue  # Skip if already processed
                    
                merged = False
                if parent_path:
                    # Find other children of the same parent that aren't already assigned
                    potential_siblings = [
                        (p, mem)
                        for p, mem, lvl, par, _ in clusters_with_level
                        if par == parent_path and p != path and not any(m in assigned_labels for m in mem)
                    ]
                    
                    for sibling_path, sibling_members in potential_siblings:
                        if len(members) + len(sibling_members) <= max_size:
                            merged_members = members.union(sibling_members)
                            
                            # print(f"Merging small cluster with parent sibling: {path} and {sibling_path}")
                            
                            assigned_color = color_list[color_idx]
                            color_idx = (color_idx + 1) % len(color_list)
                            
                            cluster_colors[tuple(sorted(members))] = assigned_color
                            cluster_colors[tuple(sorted(sibling_members))] = assigned_color
                            cluster_colors[tuple(sorted(merged_members))] = assigned_color
                            
                            if level not in merged_by_level:
                                merged_by_level[level] = []
                            merged_by_level[level].append(list(merged_members))
                            
                            assigned_labels.update(merged_members)
                            merged = True
                            merged_any = True
                            break
                
                if not merged:
                    # Sort levels from lowest to current to try merging with most hierarchically similar first
                    merger_levels = sorted([l for l in merged_by_level.keys()], key=lambda x: abs(x - level))
                    
                    for target_level in merger_levels:
                        for idx, merged_cluster_list in enumerate(merged_by_level[target_level]):
                            merged_cluster = set(merged_cluster_list)
                            if len(merged_cluster) + len(members) <= max_size:
                                # print(f"Merging small cluster with existing merged cluster: {path}")
                                
                                # Preserve color of the existing merged cluster
                                existing_cluster_key = tuple(sorted(merged_cluster_list))
                                if existing_cluster_key in cluster_colors:
                                    assigned_color = cluster_colors[existing_cluster_key]
                                else:
                                    assigned_color = color_list[color_idx]
                                    color_idx = (color_idx + 1) % len(color_list)
                                
                                # Update cluster
                                merged_cluster.update(members)
                                merged_by_level[target_level][idx] = list(merged_cluster)
                                
                                # print(f"Updated merged cluster: {merged_cluster}")
                                
                                # Assign colors
                                cluster_colors[tuple(sorted(members))] = assigned_color
                                cluster_colors[tuple(sorted(merged_cluster))] = assigned_color
                                
                                assigned_labels.update(members)
                                merged = True
                                merged_any = True
                                break
                        
                        if merged:
                            break
                
                if not merged:
                    if not force_merge_all_small:
                        if level not in merged_by_level:
                            merged_by_level[level] = []
                        merged_by_level[level].append(list(members))
                        
                        assigned_color = color_list[color_idx]
                        color_idx = (color_idx + 1) % len(color_list)
                        cluster_colors[tuple(sorted(members))] = assigned_color
                    else:
                        all_merger_candidates = []
                        
                        for l, clusters in merged_by_level.items():
                            for idx, cluster_list in enumerate(clusters):
                                cluster = set(cluster_list)
                                if len(cluster) + len(members) <= max_size:
                                    distance = abs(l - level)
                                    all_merger_candidates.append((distance, l, idx, cluster))
                        
                        all_merger_candidates.sort()
                        
                        if all_merger_candidates:
                            distance, target_level, idx, target_cluster = all_merger_candidates[0]
                            
                            print(f"FORCED MERGE: Small cluster {path} with level {target_level} cluster")
                            
                            existing_cluster_key = tuple(sorted(target_cluster))
                            if existing_cluster_key in cluster_colors:
                                assigned_color = cluster_colors[existing_cluster_key]
                            else:
                                assigned_color = color_list[color_idx]
                                color_idx = (color_idx + 1) % len(color_list)
                            
                            target_cluster.update(members)
                            merged_by_level[target_level][idx] = list(target_cluster)
                            
                            cluster_colors[tuple(sorted(members))] = assigned_color
                            cluster_colors[tuple(sorted(target_cluster))] = assigned_color
                            
                            assigned_labels.update(members)
                            merged = True
                            merged_any = True
                        else:
                            if level not in merged_by_level:
                                merged_by_level[level] = []
                            merged_by_level[level].append(list(members))
                            
                            assigned_color = color_list[color_idx]
                            color_idx = (color_idx + 1) % len(color_list)
                            cluster_colors[tuple(sorted(members))] = assigned_color
        
        if not merged_any:
            print("No merges possible. Some clusters may still be below minimum size.")
            break
    
        clusters_with_level = []
        
        for level, clusters in merged_by_level.items():
            for cluster_members in clusters:
                dummy_path = f"merged_level_{level}_{len(clusters_with_level)}"
                clusters_with_level.append((dummy_path, set(cluster_members), level, None, []))
    
    final_merged_by_level = {}
    for level, clusters in merged_by_level.items() :
        final_merged_by_level[level] = clusters
    
    small_clusters_remaining = []
    for level, clusters in final_merged_by_level.items():
        for cluster in clusters:
            if len(cluster) < min_size:
                small_clusters_remaining.append((level, cluster))
            
            if small_clusters_remaining:
                print(f"WARNING: {len(small_clusters_remaining)} clusters still below minimum size after merging:")
                for level, cluster in small_clusters_remaining:
                    print(f"  Level {level}, size {len(cluster)}: {cluster}")
                
                if force_merge_all_small:
                    remaining_small = []
                    for level, cluster in small_clusters_remaining:
                        remaining_small.append((level, set(cluster)))
                    

                if small_clusters_remaining:
                    print(f"Force merging {len(small_clusters_remaining)} remaining small clusters...")

                    for level, small_cluster in small_clusters_remaining:
                        cluster_key = None
                        
                        for key, value in cluster_dict.items(): 
                            if value["members"] == set(small_cluster):
                                cluster_key = key
                                break

                        if cluster_key and "siblings" in cluster_dict[cluster_key]:  
                            siblings = cluster_dict[cluster_key]["siblings"]
                            
                            for sibling_key in siblings:
                                if sibling_key in cluster_dict:
                                    sibling_cluster = cluster_dict[sibling_key]["members"]
                                    
                                    if len(sibling_cluster) >= min_size:
                                        print(f"Merging small cluster {small_cluster} with its sibling {sibling_cluster}")
                                        
                                        small_cluster_tuple = tuple(sorted(small_cluster))
                                        sibling_cluster_tuple = tuple(sorted(sibling_cluster))

                                        if small_cluster_tuple not in cluster_colors:
                                            cluster_colors[small_cluster_tuple] = color_list[color_idx]  

                                        if sibling_cluster_tuple not in cluster_colors:
                                            cluster_colors[sibling_cluster_tuple] = cluster_colors[small_cluster_tuple] 

                                        cluster_dict[sibling_key]["members"].update(small_cluster)
                                        
                                        del cluster_dict[cluster_key]  

                                        break
    
    
    else:
        print("All clusters meet the minimum size requirement.")
    
    for level in list(final_merged_by_level.keys()):
        final_merged_by_level[level] = [cluster for cluster in final_merged_by_level[level] if cluster]
    
    clusters = sum(final_merged_by_level.values(), [])

    tree_image_path = os.path.join(output_dir, f"tree_structure_{i}_new")
    dot = odac2.draw(show_clusters_info=["timeseries_names"], cluster_colors=cluster_colors)
    dot.render(tree_image_path, format="png", cleanup=True)
    
    return clusters

In [ ]:
def get_cluster_dict(self):
    cluster_dict = {}

    for cluster in self._find_all_active_clusters(self._root_node):
        level = '_'.join(cluster.name.split('_')[1:3])  # Get "LVL_1", "LVL_2", etc.
        
        if level not in cluster_dict:
            cluster_dict[level] = []
        cluster_dict[level].append(set(cluster.timeseries_names)) 

    return cluster_dict

def sort_cluster_levels(cluster_dict): #Sorts the cluster levels numerically, based on the level number after "LVL_".
    
    def extract_level_number(cluster_name):
        if "_LVL_" in cluster_name:
            parts = cluster_name.split("_LVL_")
            level_part = parts[1].split("_")[0]  
            return int(level_part)  
        elif cluster_name == "ROOT":
            return float('inf')  
        return float('inf') 

    # Sort the levels by their numeric part, with special handling for ROOT and unknown formats
    sorted_levels = sorted(cluster_dict.keys(), key=extract_level_number, reverse=True)
    return sorted_levels

In [ ]:
#DATASTE
stream = Yeast() #  Scene() #  Hypersphere() # Yeast() # Emotions() Yeast()  Chd #-----CHANGE HERE

#----------///------

majority_labelset = MajorityLabelset()

color_list = ['#FF5733', '#33FF57', '#FF33A8', '#FF7F50', '#DAF7A6', 
              '#8A2BE2', '#7FFF00', '#D2691E', '#6495ED', '#FF1493', 
              '#00FFFF', '#FFD700', '#F08080', '#90EE90', '#FF6347']
              
#----------///------
dataset=stream
dataset_name = dataset.filename
output_dir = os.path.join("tree_plots", f"{dataset_name}")
os.makedirs(output_dir, exist_ok=True)

metrics_csv_path = os.path.join(output_dir, "final_metrics.csv")
drifts_csv_path = os.path.join(output_dir, "drifts.csv")
drifts_data = [["Observation Index", "Num clusters", "New Ensemble Nodes"]]

sample_count = dataset.n_samples
odac2 = ODAC2()

accuracy_over_time = []
drift_detection_indexes = []

X = 1
models = {i: MLHAT() for i in range(1)}

global_metric = Metrics([ExactMatch(), MacroAverage(F1()), MicroAverage(F1()), metrics.multioutput.SampleAverage(metrics.Jaccard()), metrics.multioutput.SampleAverage(metrics.Accuracy())])

individual_metrics = {i: Metrics([ExactMatch(), MacroAverage(F1()), MicroAverage(F1()), metrics.multioutput.SampleAverage(metrics.Jaccard()), metrics.multioutput.SampleAverage(metrics.Accuracy())]) for i in range(X)}

aux = 0
t = 50
i = 0
analyze_at_index = 300


ALL_LABELS = list(next(iter(dataset))[1].keys())
model_labels = {0: ALL_LABELS}

for x, y in dataset:
    i += 1
    aux += 1
    odac2.learn_one(y)

    label_sum = defaultdict(float)
    label_count = defaultdict(int)

    for model_id, model in models.items():
        relevant_labels = model_labels.get(model_id, [])
        y_filtered = {label: y[label] for label in relevant_labels if label in y}
        y_pred = model.predict_proba_one(x)
        y_pred_filtered = {label: y_pred[label] for label in relevant_labels if label in y_pred}

        if aux > t:
            y_pred_binary = {label: max(probs, key=probs.get) for label, probs in y_pred_filtered.items()}
            individual_metrics[model_id].update(y_filtered, y_pred_binary)

        for label, probs in y_pred_filtered.items():
            label_sum[label] += int(probs.get(True, 0.0) == True)
            label_count[label] += 1

        model.learn_one(x, y_filtered)

    y_pred_avg = {label: (label_sum[label] / label_count[label]) if label_count[label] > 0 else 0.0 for label in label_sum}
    if y_pred_avg:
        best_label = max(y_pred_avg, key=y_pred_avg.get)
        y_pred_binary = {label: (label == best_label) for label in y_pred_avg}
    else:
        y_pred_binary = {}

    for label in y.keys():
        if label not in y_pred_binary:
            y_pred_binary[label] = False  

    global_metric.update(y, y_pred_binary)

    exact_match_score = global_metric.get()[0]
    accuracy_over_time.append(exact_match_score)


    # Train the MajorityLabelset classifier
    majority_labelset.learn_one(x, y)


    if odac2.structure_changed:
        if i > 100:
        
            if sample_count - i < 100:  
                continue

            aux = 0
            print(f"Structure changed at observation {i}")
            drift_detection_indexes.append(i)

            cluster_labels = odac2._get_cluster_labels_string()   
            drifts_data.append([i, len(cluster_labels), cluster_labels])

            if i > 50:

                freq_table = majority_labelset.get_frequency_table()
                size_distribution = analyze_labelset_sizes(freq_table)
                safe_min_size = get_safe_min_cluster_size(size_distribution)
               
                safe_max_size=sample_count/2

                cluster_dict = odac2.get_cluster_dict()  
                sorted_levels = sort_cluster_levels(cluster_dict) 


                refined_clusters = merge_and_split_clusters(odac2, cluster_dict, safe_min_size, safe_max_size,output_dir, i, color_list)
                print("Refined Clusters:", refined_clusters)

        print("----------------")

print("Done!")
print("Global Metrics:", global_metric.get())

for model_id, metric in individual_metrics.items():
    print(f"Model {model_id} metric: {metric.get()}")

# Save CSV
metrics_data = [["Metric Type", "Exact Match", "Macro F1", "Micro F1", "Jaccard", "Sample Avg Accuracy", "Drift Detection Indexes"]]
global_metrics_values = global_metric.get()
metrics_data.append(["Global"] + global_metrics_values + [drift_detection_indexes])
  

ROOT cluster initialized
Cluster LVL_1_ROOT_0: ['Class1', 'Class10', 'Class12', 'Class2', 'Class3', 'Class4', 'Class5', 'Class8', 'Class9']
Cluster LVL_1_ROOT_1: ['Class11', 'Class13', 'Class14', 'Class6', 'Class7']
----------------
Cluster LVL_2_ROOT_0_0: ['Class1', 'Class10', 'Class12', 'Class3', 'Class8', 'Class9']
Cluster LVL_2_ROOT_0_1: ['Class2', 'Class4', 'Class5']
Cluster LVL_2_ROOT_1_0: ['Class13', 'Class6']
Cluster LVL_2_ROOT_1_1: ['Class11', 'Class14', 'Class7']
Structure changed at observation 200
Labelset Size Distribution: {4: 91, 2: 36, 3: 7, 5: 12, 7: 14, 6: 35, 8: 4, 1: 1}
Recommended Minimum Cluster Size: 4
Starting iteration 1
Found 3 small clusters in iteration 1
Merging true sibling clusters: LVL_2_ROOT_1_0 and LVL_2_ROOT_1_1
Merging small cluster with parent sibling: LVL_2_ROOT_0_1 and LVL_2_ROOT_0_0
Starting iteration 2
All clusters meet the minimum size requirement.
Refined Clusters: [['Class6', 'Class7', 'Class13', 'Class14', 'Class11'], ['Class4', 'Class1', 'C